In [1]:
import requests
import datetime
from urllib.parse import urlencode

In [2]:
import base64

In [3]:
client_id = "c060a698be6449279c07f9f4a3c6e952"
client_secret = "92f9c0d577024d3e85ee90482b268e19"

In [4]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id =self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return{
            "Authorization":f"Basic {client_creds_b64}" 
        }
    
    def get_token_data(self):
        return {
        "grant_type":"client_credentials"
        }
    
    def perform_auth(self):
        # do a lookup for a token
        # this token is for future requests
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url,data=token_data,headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("Could not authenticate client.")
            #return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token'] # state-like item(they expire with time)
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
    def get_resource(self,lookup_id,resource_type="albums",version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def base_search(self,query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def search(self,query=None,operator=None,operator_query=None,search_type='artist'):
        if query == None:
            raise Exception("A query is required.")
        if isinstance(query,dict):
            query = " ".join([f"{k}:{v}" for k ,v in query.items()])
        if operator != None and operator_query ==  None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query,str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query,"type":search_type.lower()})
        print(query_params)
        return self.base_search(query_params)
            
        
    
    def get_album(self,_id):
        return self.get_resource(_id,resource_type='albums')
    
    def get_artist(self,_id):
        return self.get_resource(_id,resource_type='artists')
  

In [5]:
spotify = SpotifyAPI(client_id,client_secret)

In [6]:
spotify.perform_auth()

True

In [7]:
access_token = spotify.access_token
access_token

'BQD4UxWpxOMXWPi0MSCK9z4QdyEuBftbBHhxI8tvzTPPpORY1GlzAeNgyFXhjlPYQDNJx-AsKnSLvygcPeI'

In [8]:
#spotify.get_artist("137W8MRPWKqSmrBGDBFSop")

In [9]:
#spotify.get_album("2Ti79nwTsont5ZHfdxIzAm")

In [10]:
spotify.search({"track":"Hope","artist":"Twista"}, search_type="track")

q=track%3AHope+artist%3ATwista&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AHope+artist%3ATwista&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5NDMothbpdpq2xHqSjrrWn'},
       'href': 'https://api.spotify.com/v1/artists/5NDMothbpdpq2xHqSjrrWn',
       'id': '5NDMothbpdpq2xHqSjrrWn',
       'name': 'Faith Evans',
       'type': 'artist',
       'uri': 'spotify:artist:5NDMothbpdpq2xHqSjrrWn'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',

In [11]:
spotify.search(query="Danger", operator='NOT', operator_query='Zone',search_type='track')

q=Danger&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Danger&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR'},
       'href': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR',
       'id': '66CXWjxzNUsdJxJ2JdwvnR',
       'name': 'Ariana Grande',
       'type': 'artist',
       'uri': 'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IS',
      '